# Lincs data extraction and get DE

1. Get LINCS data from GSE92742 and GSE70138
2. Use Limma to get differential gene expression 

In [ ]:
import pandas as pd
import numpy as np
import sys
from cmapPy.pandasGEXpress.parse import parse 
from scanpy import AnnData
import sys
import scanpy as sc
import joblib 

## step 1: Extract data and save as adata 

In [ ]:
# once get data from GSE
path = 'lincs_data/'
gene_info = pd.read_csv(path + 'GSE92742/GSE92742_Broad_LINCS_gene_info.txt', delimiter = '\t')
sample_info = pd.read_csv(path + 'GSE70138/GSE70138_Broad_LINCS_inst_info_2017-03-06.txt', delimiter = '\t')
sample_sub = sample_info[sample_info.pert_type.isin(['trt_cp', 'ctl_vehicle'])]
adata_kaggle = sc.read('../data/kaggle_train_de.h5ad')
sample_sub_sub = sample_info[sample_info.pert_iname.str.lower().isin(adata_kaggle.obs.sm_name.str.lower().unique())]
landmark_only_gctoo  = parse(path + 'GSE70138/GSE70138_Broad_LINCS_Level2_GEX_n345976x978.gctx', cid = sample_sub.inst_id)


adata = AnnData(landmark_only_gctoo.data_df.T)
adata.var_names = gene_info["pr_gene_symbol"][gene_info["pr_is_lm"] == "1"]
# get cid information 

adata.obs = sample_sub
adata.write_h5ad('data_aft_process/GSE7_lincs_level2.h5ad')


In [ ]:
# clean the data 
adata = sc.read('data_aft_process/GSE7_lincs_level2.h5ad')
adata.obs['rep_label'] = adata.obs['det_plate'].apply(lambda x: 1 if x.split('.')[0] == 'REP' else 0)
adata_left = adata[adata.obs['rep_label'] == 0]
kaggle_drugs_dic = joblib.load('../data/name_map_kaggle2linc_dic.joblib')
overlap = set(adata_left.obs.pert_iname).intersection(list(kaggle_drugs_dic.values()))
drugs_to_found = set(list(kaggle_drugs_dic.values())) - overlap
drugs_to_found_l = list(drugs_to_found) 
adata_sub = adata[adata.obs.pert_iname.isin(drugs_to_found_l)]
adata_control = adata[(adata.obs.det_plate.isin(adata_sub.obs.det_plate.unique()))&(adata.obs.pert_iname == 'DMSO')]
adata_raw = adata_left.concatenate(adata_sub)
adata_raw.obs = adata_raw.obs.drop('rep_label', axis = 1)
adata_raw.var_names = adata_left.var_names
adata_raw.obs['pert_cell_dose'] = adata_raw.obs.pert_iname.astype(str) + '@' +  \
    adata_raw.obs.cell_id.astype(str) \
        + '@' + adata_raw.obs.pert_dose.astype(str)
adata_raw.obs['pert_dosage'] = adata_raw.obs.pert_iname.astype(str) + '@' \
      + adata_raw.obs.pert_dose.astype(str)
df = adata_raw.obs.groupby('pert_cell_dose')['cell_id'].count() 
no_rep_pert_cell_dose = list(df[df!=1].index) 
print(len(df))
print(len(no_rep_pert_cell_dose))
adata_raw = adata_raw[adata_raw.obs.pert_cell_dose.isin(no_rep_pert_cell_dose)]
adata_raw_treat = adata_raw[adata_raw.obs.pert_cell_dose != 'DMSO@A375@-666.0']
print(len(adata_raw_treat.obs.pert_cell_dose.unique()))
adata_raw.write('lincs_gse7_level2.h5ad')

## Step2: get Differential gene expression of lincs data

In [ ]:
import scanpy as sc
from tqdm import tqdm 
import os 
import os, binascii, gc

adata_raw = sc.read('lincs_gse7_level2.h5ad')
compound_name_col = 'pert_dosage'
cell_types = adata_raw.obs['cell_id'].unique()
control_compound = 'DMSO@-666.0'
with open('data_2_r/cell_names.dat', 'w') as out_f:
    out_f.write('\n'.join(cell_types))
out_f.close()
with open('data_2_r/r_code.sh', 'w') as f:

    for rename_cell_type in tqdm(cell_types, position=1, desc='cell_id'):
        cell_type_working_dir = 'cell_type_{}'.format(rename_cell_type)

        os.makedirs('data_2_r/' + cell_type_working_dir, exist_ok=True)

        cell_type_selection = adata_raw.obs['cell_id'].eq(rename_cell_type)
        cell_type_bulk_adata = adata_raw[cell_type_selection].copy()

        rpert_mapping = cell_type_bulk_adata.obs[compound_name_col].drop_duplicates() \
        .reset_index(drop=True).reset_index() \
        .set_index(compound_name_col)['index'].to_dict()

        cell_type_bulk_adata.obs['Rpert'] = cell_type_bulk_adata.obs.apply(
            lambda row: rpert_mapping[row[compound_name_col]], 
            axis='columns',
        ).astype('str')

        compound_name_to_Rpert = cell_type_bulk_adata.obs.set_index(compound_name_col)['Rpert'].to_dict()
        ref_pert = compound_name_to_Rpert[control_compound]

        # save h5ad for each cell type
        cell_type_bulk_adata.write_h5ad(os.path.join('data_2_r/' + cell_type_working_dir, 'input.h5ad'))

        random_string = binascii.b2a_hex(os.urandom(15)).decode()

        gc.collect()

        
        Rscript = '`which Rscript`'

        # for limma fit
        exec_path_fit = 'kaggle/working/neurips-2023-scripts/limma_fit.r'
        design_fit = '~0+Rpert+det_plate+det_well'

        input_path = os.path.join('kaggle/input/lincs-data-v2/data_2_r/'+ cell_type_working_dir, 'input.h5ad')
        fit_output_path = os.path.join('kaggle/v2/' + cell_type_working_dir, 'limma.rds')
        fit_plot_output_path = os.path.join('kaggle/v2/' +cell_type_working_dir, 'voom.pdf')


        r_code_for_fit_in_bash = f'''
echo '# limma fit for cell type {rename_cell_type}'
{Rscript} \
{exec_path_fit} \
--input_h5ad \
{input_path} \
--design \
'{design_fit}' \
--fit_output_path \
{fit_output_path} \
--plot_output_path \
{fit_plot_output_path} 
'''

        print (r_code_for_fit_in_bash, file=f)

        # for limma contrast
        exec_path_contrast = 'kaggle/working/neurips-2023-scripts/limma_contrast.r'

        for pert in cell_type_bulk_adata.obs['Rpert'].unique():
            if pert == ref_pert:
                continue
            else:
                contrast_output_path = os.path.join('kaggle/v2/' + cell_type_working_dir, 'pert_{}_contrast_result.csv'.format(pert))
                contrast = 'Rpert'+pert+'-Rpert'+ref_pert
                r_code_for_contrast_in_bash = f'''
echo '# limma contrast for cell type {rename_cell_type} with Rpert={pert}'
{Rscript} \
{exec_path_contrast} \
--input_fit \
{fit_output_path} \
--contrast \
{contrast} \
--contrast_output_path \
{contrast_output_path}
'''

            print (r_code_for_contrast_in_bash, file=f)

In [ ]:
# ! cd v3
from csv import reader
cell_rename_all = []
with open('../input/lincs-data-v2/data_2_r/cell_names.dat') as out_f:
    for el in reader(out_f):
        cell_rename_all.append(el[0])

## mkdirs
for rename_cell_type in cell_rename_all:
    cell_type_working_dir = 'cell_type_{}'.format(rename_cell_type)
    os.makedirs(cell_type_working_dir, exist_ok=True)

## replaced the input path for kaggle
## replaced the input path for kaggle
! sed 's!--input_h5ad cell_type!--input_h5ad kaggle/input/lincs-data-v2/data_2_r/cell_type!' kaggle/input/lincs-data-v2/data_2_r/r_code.sh > ./r_code.sh

! bash r_code.sh

# Merge the data 

In [ ]:
# this code is devoted to merge the data obtained from limma 
# %%
import os, gc

from glob import glob
from tqdm.auto import tqdm
from itertools import product
import sys 

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import anndata as ad
import scanpy as sc

sys.path.append(os.path.join(os.path.dirname(__file__), 'kaggle/input/lincs-data-v2'))
# %%

pert_de_dfs = []
from csv import reader
cell_rename_all = []
with open('kaggle/input/lincs-data-v2/data_2_r/cell_names.dat') as out_f:
    for el in reader(out_f):
        cell_rename_all.append(el[0])
de_pert_cols = [
    'pert_iname',
    'pert_cell_dose',
    'cell_id',
    'pert_dose',
]
# %%
for rename_cell_type in tqdm(cell_rename_all, position=1, desc='cell_type', leave=True):
    cell_type_working_dir = 'cell_type_{}'.format(rename_cell_type)
    cell_type_bulk_adata = sc.read_h5ad(f'kaggle/input/lincs-data-v2/data_2_r/{cell_type_working_dir}/input.h5ad')
    for file in tqdm(glob(f'kaggle/v2/{cell_type_working_dir}/pert_*_contrast_result.csv'), position=2, desc='pert', leave=False):
        pert_de_df = pd.read_csv(file)
        pert_de_df = pert_de_df.rename({pert_de_df.columns[0]: 'gene'}, axis=1)

        pert = os.path.basename(file).split('_')[1]
        pert_de_df['Rpert'] = pert

        pert_obs = cell_type_bulk_adata.obs[cell_type_bulk_adata.obs['Rpert'].eq(pert)]
        for col in de_pert_cols:
            pert_de_df[col] = pert_obs[col].unique()[0]

        pert_de_dfs.append(pert_de_df)

de_df_v2 = pd.concat(pert_de_dfs)
# %%
de_df_v2.to_csv('kaggle/input/lincs-data-v2/de_df.csv')
# %%
def convert_de_df_to_anndata(de_df, pert_cols, de_sig_cutoff):
    de_df = de_df.copy()
    zero_pval_selection = de_df['P.Value'].eq(0)
    de_df.loc[zero_pval_selection, 'P.Value'] = np.finfo(np.float64).eps

    de_df['sign_log10_pval'] = np.sign(de_df['logFC']) * -np.log10(de_df['P.Value'])
    de_df['is_de'] = de_df['P.Value'].lt(de_sig_cutoff)
    de_df['is_de_adj'] = de_df['adj.P.Val'].lt(de_sig_cutoff)

    de_feature_dfs = {}
    for feature in tqdm(['is_de', 'is_de_adj', 'sign_log10_pval', 'logFC', 'P.Value', 'adj.P.Val']):
        df = de_df.reset_index().pivot_table(
            index=['gene'], 
            columns=pert_cols,
            values=[feature],
            dropna=True,
        )
        de_feature_dfs[feature] = df


    tdf = de_feature_dfs['sign_log10_pval'].T.reset_index().drop(columns=['level_0'])
    tdf = tdf.astype({'pert_dose':str}).set_index(['pert_iname', 'pert_dose', 'cell_id', 'pert_cell_dose'])
    de_adata = ad.AnnData(
        tdf.values,
        obs=pd.DataFrame(tdf.index.to_frame().values, columns=tdf.index.names),
        var=tdf.columns.to_frame(),
        dtype=np.float64
    )

    de_adata.layers['is_de'] = de_feature_dfs['is_de'].to_numpy().T
    de_adata.layers['is_de_adj'] = de_feature_dfs['is_de_adj'].to_numpy().T
    de_adata.layers['logFC'] = de_feature_dfs['logFC'].to_numpy().T
    de_adata.layers['P.Value'] = de_feature_dfs['P.Value'].to_numpy().T
    de_adata.layers['adj.P.Val'] = de_feature_dfs['adj.P.Val'].to_numpy().T
    de_adata.obs.index = de_adata.obs.index.astype('str')
    
    return de_adata

# %%
de_adata_v2 = convert_de_df_to_anndata(de_df_v2, de_pert_cols, 0.05)
sorting_index = de_adata_v2.obs.astype(str).sort_values(['pert_iname', 'cell_id']).index
de_adata_v2 = de_adata_v2[sorting_index].copy()
de_adata_v2.write_h5ad('kaggle/input/lincs-data-v2/de_adata_lincs_limma.h5ad')